In [2]:
!pip install lightgbm

You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn import model_selection
from sklearn.metrics import roc_auc_score

In [ ]:
data = pd.read_csv("../data/titanic_proc.csv", index_col="PassengerId")

In [10]:
with open(r"C:/Users/Rafael/Documents/data/clarin_clase_210906/paywall_v.csv") as f:
    for i, l in enumerate(f):
        print(l)
        if i > 5:
            break
        

pase_id;dias_navegacion_x;n_visitas_x;n_disp_x;v_pw_x;q_notas_x;q_homes_x;pv_deportes_x;pv_espectaculos_x;pv_estilo_x;pv_noticias_x;pv_servicios_x;pv_sudoku_x;pv_sociedad_x;a_edad;antiguedad_pase;q_visitas;genero_n;Segmento;susc_target

214;4;1;1;0;3;4;0;0;0;0;0;0;15;30;2288;12;0;1;0

337;5;1;1;0;3;829;0;2;0;7;0;0;2;53;2286;12;1;2;0

345;3;1;1;0;3;1;0;3;0;1;0;0;2;64;2286;8;0;1;0

364;9;1;1;6;26;919;143;4;0;63;0;4;54;40;2283;11;0;2;0

368;8;1;1;1;20;16;0;45;8;20;0;0;49;19;2283;12;0;1;0

388;1;1;1;0;0;0;1;0;0;0;0;0;0;71;2277;7;1;1;0



In [5]:
data = pd.read_csv(r"C:/Users/Rafael/Documents/data/clarin_clase_210906/paywall_v.csv", index_col="pase_id", sep=";")

In [36]:
for c in data.select_dtypes("object").columns:
    data[c] = data[c].astype("category")

In [17]:
data.dtypes

dias_navegacion_x    int64
n_visitas_x          int64
n_disp_x             int64
v_pw_x               int64
q_notas_x            int64
q_homes_x            int64
pv_deportes_x        int64
pv_espectaculos_x    int64
pv_estilo_x          int64
pv_noticias_x        int64
pv_servicios_x       int64
pv_sudoku_x          int64
pv_sociedad_x        int64
a_edad               int64
antiguedad_pase      int64
q_visitas            int64
genero_n             int64
Segmento             int64
susc_target          int64
dtype: object

In [14]:
target = "susc_target"
# target = "Survived"
X_train, X_test, y_train, y_test = model_selection.train_test_split(data.drop(target, axis=1), 
                                                                    data[target], test_size=0.1, random_state=0)

In [20]:
kf = model_selection.KFold(n_splits=5)
folds = [(y_train.index[train_idx], 
          y_train.index[valid_idx]) for train_idx, valid_idx 
         in kf.split(X_train)]

In [21]:
valid_probs = []
test_probs = []
for i, (train_idx, valid_idx) in enumerate(folds):
    Xt = X_train.loc[train_idx]
    yt = y_train.loc[train_idx]

    Xv = X_train.loc[valid_idx]
    yv = y_train.loc[valid_idx]

    learner = LGBMClassifier(n_estimators=10000)
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt),
                          (Xv, yv)])
    probs = pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index)
    valid_probs.append(probs)
    probs = pd.Series(learner.predict_proba(X_test)[:, -1],
                      index=X_test.index, name="fold_" + str(i))
    test_probs.append(probs)
valid_probs = pd.concat(valid_probs)
test_probs = pd.concat(test_probs, axis=1).mean(axis=1)

[1]	training's auc: 0.730178	training's binary_logloss: 0.0109918	valid_1's auc: 0.692216	valid_1's binary_logloss: 0.0105291
Training until validation scores don't improve for 10 rounds.
[2]	training's auc: 0.718161	training's binary_logloss: 0.0103142	valid_1's auc: 0.698548	valid_1's binary_logloss: 0.00909039
[3]	training's auc: 0.737981	training's binary_logloss: 0.010242	valid_1's auc: 0.705375	valid_1's binary_logloss: 0.00934371
[4]	training's auc: 0.745272	training's binary_logloss: 0.0102647	valid_1's auc: 0.703256	valid_1's binary_logloss: 0.00966249
[5]	training's auc: 0.754321	training's binary_logloss: 0.0100994	valid_1's auc: 0.706547	valid_1's binary_logloss: 0.00961131
[6]	training's auc: 0.759987	training's binary_logloss: 0.010078	valid_1's auc: 0.703326	valid_1's binary_logloss: 0.00979764
[7]	training's auc: 0.765731	training's binary_logloss: 0.0099601	valid_1's auc: 0.699018	valid_1's binary_logloss: 0.00984366
[8]	training's auc: 0.771271	training's binary_loglo

[9]	training's auc: 0.797198	training's binary_logloss: 0.0100248	valid_1's auc: 0.705502	valid_1's binary_logloss: 0.011651
[10]	training's auc: 0.799744	training's binary_logloss: 0.00977853	valid_1's auc: 0.701303	valid_1's binary_logloss: 0.0116973
[11]	training's auc: 0.803393	training's binary_logloss: 0.00946772	valid_1's auc: 0.696588	valid_1's binary_logloss: 0.0114534
[12]	training's auc: 0.80644	training's binary_logloss: 0.00929928	valid_1's auc: 0.694052	valid_1's binary_logloss: 0.0113664
Early stopping, best iteration is:
[2]	training's auc: 0.741153	training's binary_logloss: 0.00990262	valid_1's auc: 0.716175	valid_1's binary_logloss: 0.0103384


In [22]:
roc_auc_score(y_test, test_probs)

0.7362436957609646

In [31]:
testeo = test_probs.rename("prob").to_frame().join(data[target])
pd.crosstab(pd.qcut(testeo.prob, 5), testeo[target])

susc_target,0,1
prob,,
"(-0.00017900000000000001, 0.00108]",18554,2
"(0.00108, 0.00115]",14236,9
"(0.00115, 0.00123]",15360,15
"(0.00123, 0.00142]",15831,27
"(0.00142, 0.398]",15910,49


In [34]:
pd.Series(learner.feature_importances_, index=Xt.columns).sort_values()

n_visitas_x           0
pv_servicios_x        0
genero_n              0
Segmento              1
q_visitas             1
pv_sudoku_x           2
pv_noticias_x         2
dias_navegacion_x     2
n_disp_x              2
pv_deportes_x         3
pv_espectaculos_x     3
q_notas_x             4
pv_sociedad_x         4
pv_estilo_x           4
v_pw_x                6
a_edad                7
antiguedad_pase       7
q_homes_x            12
dtype: int32

In [37]:
pd.crosstab(data.q_homes_x, data[target]).apply(lambda x: x / x.sum(), axis=1)

susc_target,0,1
q_homes_x,,
0,0.999431,0.000569
1,0.999253,0.000747
2,0.999003,0.000997
3,0.999056,0.000944
4,0.998652,0.001348
5,0.998365,0.001635
6,0.998982,0.001018
7,0.999300,0.000700
8,0.998543,0.001457
